# 本題要找出那些user可能會取消合約 (\#Join, \#GroupBy, \#Partption, \#Datediff )
原題目連結: https://leetcode.com/problems/find-churn-risk-customers/description/  

Table: `subscription_events`  
```
+------------------+---------+
| Column Name      | Type    | 
+------------------+---------+
| event_id         | int     |
| user_id          | int     |
| event_date       | date    |
| event_type       | varchar |
| plan_name        | varchar |
| monthly_amount   | decimal |
+------------------+---------+
```

- event_id is the unique identifier for this table.  
- event_type can be start, upgrade, downgrade, or cancel.  
- plan_name can be basic, standard, premium, or NULL (when event_type is cancel).  
- monthly_amount represents the monthly subscription cost after this event.  
- For cancel events, monthly_amount is 0.

Write a solution to Find Churn Risk Customers - users who show warning signs before churning. A user is considered churn risk customer if they meet ALL the following criteria:  

- Currently have an active subscription (their last event is not cancel).  
- Have performed at least one downgrade in their subscription history.  
- Their current plan revenue is less than 50% of their historical maximum plan revenue.  
- Have been a subscriber for at least 60 days.  

Return the result table ordered by days_as_subscriber in descending order, then by user_id in ascending order.  
The result format is in the following example.  

範例:  
Ex1:  
Input:  
`subscription_events` table:  
```
+----------+---------+------------+------------+-----------+----------------+
| event_id | user_id | event_date | event_type | plan_name | monthly_amount |
+----------+---------+------------+------------+-----------+----------------+
| 1        | 501     | 2024-01-01 | start      | premium   | 29.99          |
| 2        | 501     | 2024-02-15 | downgrade  | standard  | 19.99          |
| 3        | 501     | 2024-03-20 | downgrade  | basic     | 9.99           |
| 4        | 502     | 2024-01-05 | start      | standard  | 19.99          |
| 5        | 502     | 2024-02-10 | upgrade    | premium   | 29.99          |
| 6        | 502     | 2024-03-15 | downgrade  | basic     | 9.99           |
| 7        | 503     | 2024-01-10 | start      | basic     | 9.99           |
| 8        | 503     | 2024-02-20 | upgrade    | standard  | 19.99          |
| 9        | 503     | 2024-03-25 | upgrade    | premium   | 29.99          |
| 10       | 504     | 2024-01-15 | start      | premium   | 29.99          |
| 11       | 504     | 2024-03-01 | downgrade  | standard  | 19.99          |
| 12       | 504     | 2024-03-30 | cancel     | NULL      | 0.00           |
| 13       | 505     | 2024-02-01 | start      | basic     | 9.99           |
| 14       | 505     | 2024-02-28 | upgrade    | standard  | 19.99          |
| 15       | 506     | 2024-01-20 | start      | premium   | 29.99          |
| 16       | 506     | 2024-03-10 | downgrade  | basic     | 9.99           |
+----------+---------+------------+------------+-----------+----------------+
```
Output:  
```
+----------+--------------+------------------------+-----------------------+--------------------+
| user_id  | current_plan | current_monthly_amount | max_historical_amount | days_as_subscriber |
+----------+--------------+------------------------+-----------------------+--------------------+
| 501      | basic        | 9.99                   | 29.99                 | 79                 |
| 502      | basic        | 9.99                   | 29.99                 | 69                 |
+----------+--------------+------------------------+-----------------------+--------------------+
```
Explanation:  

- User 501:  
Currently active: Last event is downgrade to basic (not cancelled)   
Has downgrades: Yes, 2 downgrades in history   
Current revenue (9.99) vs max (29.99): 9.99/29.99 = 33.3% (less than 50%)   
Days as subscriber: Jan 1 to Mar 20 = 79 days (at least 60)   
Result: Churn Risk Customer  

- User 502:  
Currently active: Last event is downgrade to basic (not cancelled)   
Has downgrades: Yes, 1 downgrade in history   
Current revenue (9.99) vs max (29.99): 9.99/29.99 = 33.3% (less than 50%)   
Days as subscriber: Jan 5 to Mar 15 = 70 days (at least 60)   
Result: Churn Risk Customer


- User 503:  
Currently active: Last event is upgrade to premium (not cancelled)   
Has downgrades: No downgrades in history   
Result: Not at-risk (no downgrade history)

- User 504:  
Currently active: Last event is cancel  
Result: Not at-risk (subscription cancelled)

- User 505:  
Currently active: Last event is 'upgrade' to standard (not cancelled)   
Has downgrades: No downgrades in history   
Result: Not at-risk (no downgrade history)

- User 506:  
Currently active: Last event is downgrade to basic (not cancelled)   
Has downgrades: Yes, 1 downgrade in history   
Current revenue (9.99) vs max (29.99): 9.99/29.99 = 33.3% (less than 50%)   
Days as subscriber: Jan 20 to Mar 10 = 50 days (less than 60)   
Result: Not at-risk (insufficient subscription duration)

Result table is ordered by days_as_subscriber DESC, then user_id ASC.  
Note: days_as_subscriber is calculated from the first event date to the last event date for each user.  

* 解題想法:  
首先找出有進行過downgrade的user_id，並與做了partition之後的subscription_events進行join，接著找出每個user_id的合約時間以及最大的合約金額後進行join，最後過濾合約時間最少六十天，最後一筆資料必須不是cancel，以及目前合約金額必須為最大合約的50%以下，就可以找出指定的資料

```
# Write your MySQL query statement below

select user_id, plan_name as current_plan, monthly_amount as current_monthly_amount, amount as max_historical_amount, d as days_as_subscriber from 
(select * from 
(select d.user_id, d.event_date, d.event_type, d.plan_name, d.monthly_amount, d.o, day.d, day.amount from 
( select *, row_number() over (partition by user_id order by event_date desc) as o from subscription_events 
right join 
( select distinct(user_id) as u from subscription_events where event_type = 'downgrade' ) down
on subscription_events.user_id = down.u ) d
left join 
(   
    select user_id, datediff(max(event_date), min(event_date)) as d, max(monthly_amount) as amount from subscription_events group by user_id
)
day
on d.user_id = day.user_id ) agg
where o = 1 and d >= 60 and event_type != 'cancel' ) temp
where monthly_amount / amount < 0.5
order by d desc, user_id 
```